In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Manage Asset Frameworks
## Create Client
To access and edit asset frameworks we will need a client with credentials from a user with admin rights.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient
from confighub_interface import ConfigClient


url = "https://cs.trendminer.net"
client_id = "wdanielsclient"
username = "wdaniels"


client = TrendMinerClient(
    url=url, 
    client_id=client_id, 
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username)
)

## Creation
For info on the required asset framework structure: https://trendminer.elevio.help/en/articles/220

In [3]:
import pandas as pd

af_name = "MyNewAF"

# Simple example with single asset
df = pd.DataFrame(
    columns=["id", "path", "name", "description", "type", "template", "tag", "source"],
    data=[["", "/MyAsset", "MyAsset", "My favorite asset", "ASSET", "Template1", "", ""]],
)

af = client.asset.framework(name=af_name, df=df, published=False)
af.post()

af.wait_for_sync()

## Loading
Rather than creating a new framework, we can load an existing one. As an example let's load the one we just created.

In [4]:
af = client.asset.framework.from_name(af_name)

# export the current framework as a pandas DataFrame (to af.df)
af.export()

# display the dataframe
af.df

id      path     name        description  \
0  f6f5c0ac-8ffa-4929-96c1-31b91989cb67  /MyAsset  MyAsset  My favorite asset   

    type   template  tag                                source  
0  ASSET  Template1  NaN  b1c2f46a-cc28-460e-b576-74256195d654

## Editing
We can edit both the metadata and the structure for an asset framework we loaded.

In [5]:
import time


# edit the DataFrame in place (for example, end all descriptions with an exclamation mark)
af.df["description"] = af.df["description"].apply(lambda x: x+"!")

# we can also change name and the published status
af.name = af.name + "_new"
af.published = True

# To actually update the AF in TrendMiner, we need to send a put request
af.put()

# The sync will take some time, take this into account
print(f"Sync status: {af.sync.status}")

# Let's wait until the AF is updated
af.wait_for_sync()
        
# Confirm there are no errors
print(f"Sync status: {af.sync.status}")

# And show the new structure
af.df

Sync status: RUNNING
Sync status: DONE


id      path     name  \
0  f6f5c0ac-8ffa-4929-96c1-31b91989cb67  /MyAsset  MyAsset   

          description   type   template  tag  \
0  My favorite asset!  ASSET  Template1  NaN   

                                 source  
0  b1c2f46a-cc28-460e-b576-74256195d654

## Deletion
An asset framework will only get soft deleted. You will not be able to use the same name again. In case there is a possibility we want to reuse the name for a new framework, we can rename the old one before deletion, freeing up the name.

In [6]:
import datetime

unique_name = 'Del%d' % datetime.datetime.now().timestamp()
print(unique_name)

af.name = unique_name
af.put()

af.wait_for_sync()

af.delete()

Del1673274919
